In [44]:
def stem(input,stride):
    print("Shape at Stem Stage 1", input.shape)
    x = Conv2D(32, 3, strides=stride, padding='same', activation='relu')(input)
    print("Shape at Stem Stage 2", x.shape)
    x = BatchNormalization()(x)
    print("Shape at Stem Stage 3", x.shape)
    x = Conv2D(32, 3, strides=1, padding='same', activation='relu')(x)
    print("Shape at Stem Stage 4", x.shape)
    x = BatchNormalization()(x)
    print("Shape at Stem Stage 5", x.shape)
    return x

In [45]:
def lka(input,fltr,dm=1):
    print("Shape at lka Stage 1", input.shape)
    x = DepthwiseConv2D(3, strides=1, padding='same', depth_multiplier=dm, activation='relu')(input)
    print("Shape at lka Stage 2", x.shape)
    x = BatchNormalization()(x)
    print("Shape at lka Stage 3", x.shape)
    x = DepthwiseConv2D(3, strides=1, padding='same', depth_multiplier=dm, activation='relu',dilation_rate=3)(x)
    print("Shape at lka Stage 4", x.shape)
    x = BatchNormalization()(x)
    print("Shape at lka Stage 5", x.shape)
    x = Conv2D(fltr,1,padding='same',activation='relu')(x)
    print("Shape at lka Stage 6", x.shape)
    x = Multiply()([x,input])
    print("Shape at lka Stage 7", x.shape)
    return x

In [46]:
# def lka(input,fltr,dm=1):
#     x = DepthwiseConv2D(3, strides=1, padding='same', depth_multiplier=dm, activation='relu')(input)
#     x = BatchNormalization()(x)
#     x = DepthwiseConv2D(3, strides=1, padding='same', depth_multiplier=dm, activation='relu',dilation_rate=3)(x)
#     x = BatchNormalization()(x)
#     x = Conv2D(fltr,1,padding='same',activation='relu')(x)
#     x = Multiply()([x,input])
#     return x

In [55]:
def lka_attn(input,fltr,nh):
    attn = []
    act = ['relu',tf.nn.gelu,'sigmoid']
    y = input
    print("Stage 1 of LKA attn", input.shape)
    for i in range(nh):
        print("Stage",i+1, "of LKA attn", input.shape)
        x = Conv2D(fltr, 1, strides=1, padding='same', activation=act[i])(input)
        print("Stage",i+2, "of LKA attn", x.shape)
        x = lka(x,fltr)
        print("Stage",i+3, "of LKA attn", x.shape)
        x = Conv2D(fltr, 1, strides=1, padding='same', activation=act[i])(x)
        print("Stage",i+4, "of LKA attn", x.shape)
        x = Add()([x,y])
        print("Stage",i+5, "of LKA attn", x.shape)
        attn.append(x)
        print("Stage",i+6, "of LKA attn", x.shape)
    mh_lka_attn = Add()([attn[0],attn[1],attn[2]])
    print("Final Stage of LKA attn", mh_lka_attn.shape)
    return mh_lka_attn

In [48]:
def trans_enc(input,fltr,nl,nh):
    x = input
    print("Stage 1 of trans_enc attn", input.shape)
    k = 0
    for i in range(nl):
        i = k
        print("Stage",i, "of trans_enc attn", input.shape)
        x = LayerNormalization(epsilon=1e-6)(x)
        print("Stage",i+1, "of trans_enc attn", x.shape)
        x = lka_attn(x,fltr,nh)
        print("Stage",i+2, "of trans_enc attn", x.shape)
        x = Add()([input,x])
        print("Stage",i+3, "of trans_enc attn", x.shape)
        y = LayerNormalization(epsilon=1e-6)(x)
        print("Stage",i+4, "of trans_enc attn", x.shape)
        y = Conv2D(fltr, 1, strides=1, padding='same', activation='relu')(y)
        print("Stage",i+5, "of trans_enc attn", x.shape)
        x = Add()([x,y])
        print("Stage",i+6, "of trans_enc attn", x.shape)
        k = i + 6
    return x


In [49]:
def MSConv(input,fltr,strides,dm):
    shape = K.int_shape(input)
    print("Stage 1 of MSConv attn", input.shape)
    x = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(input)
    x = BatchNormalization()(x)
    print("Stage 2 of MSConv attn", x.shape)
    y = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(x)
    y = BatchNormalization()(y)
    print("Stage 3 of MSConv attn", y.shape)
    z = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(y)
    z = BatchNormalization()(z)
    print("Stage 4 of MSConv attn", z.shape)
    z = Concatenate()([x,y,z])
    print("Stage 5 of MSConv attn", z.shape)
    z = Conv2D(shape[3], 1, padding='same', activation='relu')(z)
    print("Stage 6 of MSConv attn", z.shape)
    z = Add()([input,z])
    print("Stage 7 of MSConv attn", z.shape)
    return z

In [50]:
def GAB(inputs):
    shape=K.int_shape(inputs)
    print("Stage 1 of GAB", inputs.shape)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    print("Stage 2 of GAB", x.shape)
    x=Conv2D(shape[3],1, padding='same') (x)
    print("Stage 3 of GAB", x.shape)
    x=Activation('relu') (x)
    print("Stage 4 of GAB", x.shape)
    x=Conv2D(shape[3],1, padding='same') (x)
    print("Stage 5 of GAB", x.shape)
    x=Activation('sigmoid') (x)
    print("Stage 6 of GAB", x.shape)
    C_A=Multiply()([x,inputs])
    print("Stage 7 of GAB", C_A.shape)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    print("Stage 8 of GAB", x.shape)
    x=Activation('sigmoid') (x)
    print("Stage 9 of GAB", x.shape)
    S_A=Multiply()([x,C_A])
    print("Stage 10 of GAB", S_A.shape)
    return S_A

In [51]:
def defconv(input,fltr,strides,dm=1):
    print("Stage 1 of def_conv", input.shape)
    x = BatchNormalization()(input)
    print("Stage 2 of def_conv", x.shape)
    x = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(x)
    print("Stage 3 of def_conv", x.shape)
    x = BatchNormalization()(x)
    print("Stage 4 of def_conv", x.shape)
    x = MSConv(x,fltr,strides,dm)
    print("Stage 5 of def_conv", x.shape)
    x = GAB(x)
    print("Stage 6 of def_conv", x.shape)
    x = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(x)
    print("Stage 7 of def_conv", x.shape)
    if strides==2:
        input = MaxPooling2D()(input)
        input = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(input)
        x = Add()([input,x])
    else:
        input = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(input)
        x = Add()([input,x])
    print("Stage 8 of def_conv", x.shape)
    return x

In [52]:
def Defvit_block(input, fltr,strides,nl,nh,dm=1):
    print("Stage 1 of Defvit_block", input.shape)
    x = defconv(input,fltr,strides,dm)
    print("Stage 2 of Defvit_block", x.shape)
    x = trans_enc(x,fltr,nl,nh)
    print("Stage 3 of Defvit_block", x.shape)
    return x

In [53]:
def build_defvit(input_shape,classes):
  # S-0 STEM stage
    input = Input(input_shape)
    x = stem(input,stride=2)
#     x = stem(x,stride=1)
    print('After stage-0',x.shape)

  # S-1 stage
    x = Defvit_block(x,32,strides=1,nl=2,nh=3)
    x = Defvit_block(x,32,strides=1,nl=2,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-1',x.shape)

    # S-2 stage
    x = Defvit_block(x,64,strides=1,nl=2,nh=3)
    x = Defvit_block(x,64,strides=1,nl=2,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-2',x.shape)

    # S-3 stage
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-3',x.shape)

    # S-4 stage

    x = Defvit_block(x,256,strides=1,nl=2,nh=3)
    x = Defvit_block(x,256,strides=1,nl=2,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-4',x.shape)


    x = GlobalAveragePooling2D()(x)
    out = Dense(classes,activation='softmax')(x)
    model = Model(input,out)
    return model

In [59]:
model_vit = build_defvit((224,224,3),8)

Shape at Stem Stage 1 (None, 224, 224, 3)
Shape at Stem Stage 2 (None, 112, 112, 32)
Shape at Stem Stage 3 (None, 112, 112, 32)
Shape at Stem Stage 4 (None, 112, 112, 32)
Shape at Stem Stage 5 (None, 112, 112, 32)
After stage-0 (None, 112, 112, 32)
Stage 1 of Defvit_block (None, 112, 112, 32)
Stage 1 of def_conv (None, 112, 112, 32)
Stage 2 of def_conv (None, 112, 112, 32)
Stage 3 of def_conv (None, 112, 112, 32)
Stage 4 of def_conv (None, 112, 112, 32)
Stage 1 of MSConv attn (None, 112, 112, 32)
Stage 2 of MSConv attn (None, 112, 112, 32)
Stage 3 of MSConv attn (None, 112, 112, 32)
Stage 4 of MSConv attn (None, 112, 112, 32)
Stage 5 of MSConv attn (None, 112, 112, 96)
Stage 6 of MSConv attn (None, 112, 112, 32)
Stage 7 of MSConv attn (None, 112, 112, 32)
Stage 5 of def_conv (None, 112, 112, 32)
Stage 1 of GAB (None, 112, 112, 32)
Stage 2 of GAB (None, 1, 1, 32)
Stage 3 of GAB (None, 1, 1, 32)
Stage 4 of GAB (None, 1, 1, 32)
Stage 5 of GAB (None, 1, 1, 32)
Stage 6 of GAB (None, 1, 1, 3

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
import io
from contextlib import redirect_stdout
from tensorflow.keras import Model

buffer = io.StringIO()
with redirect_stdout(buffer):
    model_vit.summary()

# Get the summary as a string
summary_str = buffer.getvalue()

# Save the summary to a text file
with open("/content/drive/MyDrive/model_summary.txt", "w") as file:
    file.write(summary_str)

print("Model summary has been saved to model_summary.txt")

Model summary has been saved to model_summary.txt
